In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import io
from PIL import Image
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from operator import itemgetter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import os
from sklearn.model_selection import KFold
import lightgbm as lgb


In [2]:
def is_kaggle():
    return os.path.exists('/kaggle')

class Config:
    BASE_PATH = '/kaggle/input/isic-2024-challenge/' if is_kaggle() else 'isic-2024-challenge/'
    TRAIN_IMAGE_PATH = 'train-image.hdf5'
    TRAIN_METADATA_PATH = 'train-metadata.csv'
    TEST_IMAGE_PATH = 'test-image.hdf5'
    TEST_METADATA_PATH = 'test-metadata.csv'
    
    # Data processing
    IMAGE_SIZE = (224, 224)
    VALIDATION_SPLIT = 0.15
    RANDOM_STATE = 42
    
    BATCH_SIZE = 32
    
    class MetadataModule:
        ACTIVATION = 'relu'
        KERNEL_INITIALIZER = 'he_normal'
        
    class ImageModule:
        ACTIVATION = 'relu'
        KERNEL_INITIALIZER = 'he_normal'

# Preprocesamiento

In [3]:
def feature_engineering(df):
    eps = 1e-6
    df["lesion_size_ratio"] = np.minimum(df["tbp_lv_minorAxisMM"] / (df["clin_size_long_diam_mm"] + eps), 1.015)
    df["lesion_shape_index"] = np.minimum(df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2 + eps), 0.093)
    df["hue_contrast"] = (df["tbp_lv_H"] - df["tbp_lv_Hext"]).abs()
    df["luminance_contrast"] = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs()
    df["lesion_color_difference"] = np.sqrt(df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2)
    df["border_complexity"] = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    df["color_uniformity"] = np.log1p(np.minimum(df["tbp_lv_color_std_mean"] / df["tbp_lv_radial_color_std_max"], 1000))
    df["3d_position_distance"] = np.sqrt(df["tbp_lv_x"] ** 2 + df["tbp_lv_y"] ** 2 + df["tbp_lv_z"] ** 2) 
    df["perimeter_to_area_ratio"] = np.minimum(df["tbp_lv_perimeterMM"] / (df["tbp_lv_areaMM2"] + eps), 6.02)
    df["lesion_visibility_score"] = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"]
    df["combined_anatomical_site"] = df["anatom_site_general"] + "_" + df["tbp_lv_location"]
    df["symmetry_border_consistency"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    df["color_consistency"] = np.minimum(df["tbp_lv_stdL"] / (df["tbp_lv_Lext"] + eps), 0.305)
    
    df["size_age_interaction"] = df["clin_size_long_diam_mm"] * df["age_approx"]
    df["hue_color_std_interaction"] = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    df["lesion_severity_index"] = (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"] + df["tbp_lv_eccentricity"]) / 3
    df["shape_complexity_index"] = df["border_complexity"] + df["lesion_shape_index"]
    df["color_contrast_index"] = df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"] + df["tbp_lv_deltaLBnorm"]
    df["log_lesion_area"] = np.log(df["tbp_lv_areaMM2"] + 1)
    df["normalized_lesion_size"] = np.minimum(df["clin_size_long_diam_mm"] / (df["age_approx"] + eps), 1.59)
    df["mean_hue_difference"] = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    df["std_dev_contrast"] = np.sqrt((df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2) / 3)
    df["color_shape_composite_index"] = (df["tbp_lv_color_std_mean"] + df["tbp_lv_area_perim_ratio"] + df["tbp_lv_symm_2axis"]) / 3
    df["3d_lesion_orientation"] = np.arctan2(df["tbp_lv_y"], df["tbp_lv_x"])
    df["overall_color_difference"] = (df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"]) / 3
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    df["comprehensive_lesion_index"] = (df["tbp_lv_area_perim_ratio"] + df["tbp_lv_eccentricity"] + df["tbp_lv_norm_color"] + df["tbp_lv_symm_2axis"]) / 4

    # Taken from: https://www.kaggle.com/code/dschettler8845/isic-detect-skin-cancer-let-s-learn-together
    df["color_variance_ratio"] = np.minimum(df["tbp_lv_color_std_mean"] / (df["tbp_lv_stdLExt"] + eps), 7.94)
    df["border_color_interaction"] = df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]
    df["size_color_contrast_ratio"] = np.minimum(df["clin_size_long_diam_mm"] / (df["tbp_lv_deltaLBnorm"] + eps), 5.08)
    df["age_normalized_nevi_confidence"] = np.minimum(df["tbp_lv_nevi_confidence"] / (df["age_approx"] + eps), 9.42)
    df["color_asymmetry_index"] = df["tbp_lv_radial_color_std_max"] * df["tbp_lv_symm_2axis"]
    df["3d_volume_approximation"] = df["tbp_lv_areaMM2"] * np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    df["color_range"] = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs() + (df["tbp_lv_A"] - df["tbp_lv_Aext"]).abs() + (df["tbp_lv_B"] - df["tbp_lv_Bext"]).abs()
    df["shape_color_consistency"] = df["tbp_lv_eccentricity"] * df["tbp_lv_color_std_mean"]
    df["border_length_ratio"] = np.minimum(df["tbp_lv_perimeterMM"] / (2 * np.pi * np.sqrt(df["tbp_lv_areaMM2"] / np.pi) + eps), 2.64)
    df["age_size_symmetry_index"] = df["age_approx"] * df["clin_size_long_diam_mm"] * df["tbp_lv_symm_2axis"]
    # Until here.
    
    new_num_cols = [
        "lesion_size_ratio", "lesion_shape_index", "hue_contrast",
        "luminance_contrast", "lesion_color_difference", "border_complexity",
        "color_uniformity", "3d_position_distance", "perimeter_to_area_ratio",
        "lesion_visibility_score", "symmetry_border_consistency", "color_consistency",

        "size_age_interaction", "hue_color_std_interaction", "lesion_severity_index", 
        "shape_complexity_index", "color_contrast_index", "log_lesion_area",
        "normalized_lesion_size", "mean_hue_difference", "std_dev_contrast",
        "color_shape_composite_index", "3d_lesion_orientation", "overall_color_difference",
        "symmetry_perimeter_interaction", "comprehensive_lesion_index",
        
        "color_variance_ratio", "border_color_interaction", "size_color_contrast_ratio",
        "age_normalized_nevi_confidence", "color_asymmetry_index", "3d_volume_approximation",
        "color_range", "shape_color_consistency", "border_length_ratio", "age_size_symmetry_index",
    ]
    new_cat_cols = ["combined_anatomical_site"]
    return df, new_num_cols, new_cat_cols

In [4]:
train_hdf5 = h5py.File(Config.BASE_PATH + Config.TRAIN_IMAGE_PATH, 'r')
test_hdf5 = h5py.File(Config.BASE_PATH + Config.TEST_IMAGE_PATH, 'r')

train_metadata = pd.read_csv(Config.BASE_PATH + Config.TRAIN_METADATA_PATH)
test_metadata = pd.read_csv(Config.BASE_PATH + Config.TEST_METADATA_PATH)

# Add features
train_metadata, new_num_cols, new_cat_cols = feature_engineering(train_metadata)
test_metadata, _, _ = feature_engineering(test_metadata)

fnames = train_metadata["isic_id"].tolist()
test_fnames = test_metadata["isic_id"].tolist()

train_target = train_metadata["target"]

split = StratifiedShuffleSplit(n_splits=1, test_size=Config.VALIDATION_SPLIT, random_state=Config.RANDOM_STATE)
for train_index, val_index in split.split(train_metadata, train_target):
    val_fnames = itemgetter(*val_index)(fnames)
    train_fnames = itemgetter(*train_index)(fnames)
    X_metadata_train, X_metadata_val = train_metadata.iloc[train_index], train_metadata.iloc[val_index]
    y_train, y_val = train_target.iloc[train_index], train_target.iloc[val_index]

/var/folders/bw/_7s8wxw93cngpt5f7bg5s3hm0000gn/T/ipykernel_34762/3660601404.py:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv(Config.BASE_PATH + Config.TRAIN_METADATA_PATH)


In [26]:
only_train_cols = ["target", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence"]
unuseful_cols = ["image_type", "patient_id"]
removable_cols = only_train_cols + unuseful_cols + ["isic_id"]

numeric_features = X_train_final.select_dtypes(include=['float64', 'int64']).columns.difference(removable_cols)
cat_features = X_train_final.select_dtypes(include=['object']).columns.difference(removable_cols)

numeric_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

cat_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', cat_pipeline, cat_features)
    ])

metadata_preprocessing_pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

X_train_metadata_preprocessed = metadata_preprocessing_pipeline.fit_transform(X_train_final)
X_val_metadata_preprocessed = metadata_preprocessing_pipeline.transform(X_val_final)
# X_test_metadata_preprocessed = metadata_preprocessing_pipeline.transform(test_metadata)


In [ ]:
def create_dataset(fnames, metadata_preprocessed, targets, hdf5):
    target_ds = tf.data.Dataset.from_tensor_slices(targets)
    
    def load_image(id):
        image = Image.open(io.BytesIO(np.array(hdf5[id.numpy()])))
        image = np.array(image.resize(Config.IMAGE_SIZE)).reshape(*Config.IMAGE_SIZE, 3)
        return image

    # It doesn't work without this in Kaggle
    def set_shapes(image):
        image.set_shape([*Config.IMAGE_SIZE, 3])
        return image

    # Create a dataset for images
    image_ds = tf.data.Dataset.from_tensor_slices(tf.constant(fnames))
    image_ds = image_ds.map(lambda x: tf.py_function(load_image, [x], tf.float32))
    image_ds = image_ds.map(set_shapes)
    solo_image_ds = tf.data.Dataset.zip((image_ds, target_ds)).batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Create a dataset for metadata
    metadata_ds = tf.data.Dataset.from_tensor_slices(metadata_preprocessed)
    solo_metadata_ds = tf.data.Dataset.zip((metadata_ds, target_ds)).batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Combine the datasets
    combined_ds = tf.data.Dataset.zip(((image_ds, metadata_ds), target_ds))
    combined_ds = combined_ds.shuffle(1000)
    combined_ds = combined_ds.batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return solo_image_ds, solo_metadata_ds, combined_ds

train_solo_image_ds, train_solo_metadata_ds, train_ds = create_dataset(train_fnames, X_train_metadata_preprocessed, y_train, train_hdf5)
val_solo_image_ds, val_solo_metadata_ds, val_ds = create_dataset(val_fnames, X_val_metadata_preprocessed, y_val, train_hdf5)

# TEST
test_solo_image_ds, test_solo_metadata_ds, test_ds = create_dataset(test_fnames, X_test_metadata_preprocessed, np.zeros(len(test_fnames)), test_hdf5)

# Auxiliary functions and classes

In [27]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

def pauc_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    v_gt = abs(y_true - 1)
    v_pred = 1.0 - y_pred
    min_tpr = 0.80
    max_fpr = 1 - min_tpr
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

class PAUCCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, batch_size):
        super(PAUCCallback, self).__init__()
        self.validation_data = validation_data
        self.batch_size = batch_size

    def on_epoch_end(self, epoch, logs=None):
        # Get predictions for validation data
        val_pred = self.model.predict(self.validation_data, verbose=0)
        
        # Extract true labels from validation data
        y_val = np.concatenate([y for x, y in self.validation_data], axis=0)
        
        # Calculate pAUC score
        pauc = pauc_score(y_val, val_pred)
        
        # Optionally, you can add the pAUC score to the logs
        logs['val_pauc'] = pauc

# Metadata module

In [28]:
features_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

X_train_metadata_preprocessed = features_pipeline.fit_transform(train_features_without_id)
X_val_metadata_preprocessed = features_pipeline.transform(val_features_without_id)

In [29]:
metadata_input_shape = next(iter(train_solo_metadata_ds.take(1)))[0].shape[1:]

shuffles = 4
splits = 2
models = []

for i in range(shuffles):
    print(f"Shuffle {i+1}/{shuffles}")
    split = KFold(n_splits=splits, shuffle=True)
    for train_index, val_index in split.split(X_train_metadata_preprocessed, y_train):
        X_metadata_train_split, X_metadata_val_split = X_train_metadata_preprocessed[train_index], X_train_metadata_preprocessed[val_index]
        y_train_split, y_val_split = y_train.iloc[train_index], y_train.iloc[val_index]
        
        lgbm_model = lgb.LGBMClassifier(
            n_estimators=500,
            max_depth=2,
            learning_rate=0.025,
            n_jobs=-1,
            verbose=-1
        )
        lgbm_model.fit(X_metadata_train_split, y_train_split)

        pred = lgbm_model.predict_proba(X_metadata_val_split)
        score = pauc_score(y_val_split, pred[:, 1])
        print(f"pAUC = {score:.4f}")
        
        models.append(lgbm_model)
    
    print("\n")

Shuffle 1/4
pAUC = 0.1133
pAUC = 0.1110


Shuffle 2/4
pAUC = 0.1042
pAUC = 0.1073


Shuffle 3/4
pAUC = 0.1196
pAUC = 0.1117


Shuffle 4/4
pAUC = 0.1072
pAUC = 0.1068




In [30]:
print("Making predictions on the validation set:")

y_preds = []

for i, model in enumerate(models, 1):
    # Make predictions on the validation set
    y_pred = model.predict_proba(X_val_metadata_preprocessed)[:, 1]
    y_preds.append(y_pred)

y_preds = np.array(y_preds)

n = 100
final_pred = np.mean(y_preds[:n], axis=0)
print(pauc_score(y_val, final_pred))

Making predictions on the validation set:
0.1183174370399616


# Image Module

In [ ]:
# image_input_shape = next(iter(train_image_ds.take(1))).shape

# image_model = Sequential([
#     Conv2D(32, 3, 2, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER, input_shape=image_input_shape),
#     Conv2D(16, 3, 2, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER),
#     MaxPooling2D(2, 2),
#     Flatten(),
#     Dense(64, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER),
#     Dense(1, activation='sigmoid')
# ])

# # Compile the model
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# image_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# # Callbacks
# pauc_callback = PAUCCallback(val_solo_image_ds.take(100), Config.BATCH_SIZE)

# # Display model summary
# image_model.summary()



In [ ]:
# image_model.fit(train_solo_image_ds.take(500), validation_data=val_solo_image_ds.take(100), epochs=1, callbacks=[pauc_callback])

# Combined Modules

In [ ]:
# image_input = tf.keras.Input(shape=image_input_shape)
# metadata_input = tf.keras.Input(shape=metadata_input_shape)

# # Clone and freeze image model layers
# # x_image = image_input
# # for layer in image_model.layers[:-1]:  # Exclude the last layer
# #     x_image = layer(x_image)
# #     layer.trainable = False

# # Clone and freeze metadata model layers
# x_metadata = metadata_input
# for layer in metadata_model.layers[:-1]:  # Exclude the last layer
#     x_metadata = layer(x_metadata)
#     layer.trainable = False

# # Concatenate the outputs of both models
# # combined = tf.keras.layers.Concatenate()([x_image, x_metadata])
# x = tf.keras.layers.Dense(16, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER)(x_metadata)
# x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# # Define inputs
# input = [image_input, metadata_input]

# combined_model = tf.keras.Model(inputs=input, outputs=x)

# pauc_callback = PAUCCallback(val_ds.take(100), Config.BATCH_SIZE)

# # Compile the model
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
# combined_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# combined_model.fit(train_ds.take(500), validation_data=val_ds.take(100), epochs=1, callbacks=[pauc_callback])

# # Unfreeze layers in the image model
# for layer in combined_model.layers:
#     if isinstance(layer, tf.keras.Model) and layer.name == image_model.name:
#         for sub_layer in layer.layers:
#             sub_layer.trainable = True

# # Unfreeze layers in the metadata model
# for layer in combined_model.layers:
#     if isinstance(layer, tf.keras.Model) and layer.name == metadata_model.name:
#         for sub_layer in layer.layers:
#             sub_layer.trainable = True

# # Recompile the model with a lower learning rate for fine-tuning
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
# combined_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# print("Layers unfrozen and model recompiled for fine-tuning.")

# combined_model.fit(train_ds.take(500), validation_data=val_ds.take(100), epochs=1, callbacks=[pauc_callback])

# Submission

In [ ]:
submission = pd.read_csv(Config.BASE_PATH + 'sample_submission.csv')

# zero_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(test_ds)))
# test_ds_with_zeros = tf.data.Dataset.zip((test_ds, zero_labels))
# zero_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(test_metadata_ds)))
# test_ds_with_zeros = tf.data.Dataset.zip((test_metadata_ds, zero_labels)).batch(Config.BATCH_SIZE)

# submission["target"] = combined_model.predict(test_ds_with_zeros)
# submission["target"] = metadata_model.predict(test_ds_with_zeros)
# submission["target"] = xgb_model.predict_proba(X_test_metadata_preprocessed)[:, 1]
y_preds = []
for i, model in enumerate(models, 1):
    # Make predictions on the validation set
    y_pred = model.predict_proba(X_test_metadata_preprocessed)[:, 1]
    y_preds.append(y_pred)

y_preds = np.array(y_preds)

final_pred = np.mean(y_preds, axis=0)
submission["target"] = final_pred
submission.to_csv('submission.csv', index=False)